<a href="https://colab.research.google.com/github/DJNahata/Style-GANS-Practice/blob/master/Style_Modulation_Function_Practice_in_NNabla_without_fused_scale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install nnabla

     |████████████████████████████████| 14.2MB 243kB/s 
     |████████████████████████████████| 4.8MB 51.2MB/s 


In [0]:
import numpy as np
import nnabla as nn
import nnabla.parametric_functions as PF
import nnabla.functions as F
import nnabla.solvers as S
import nnabla.logger as logger
import nnabla.utils.save as save
from nnabla.parameter import get_parameter_or_create
from nnabla.ext_utils import get_extension_context
from nnabla.parametric_functions import parametric_function_api
import nnabla.initializer as I
nn.set_auto_forward(True)
print("DJ1")

2020-04-21 18:36:21,314 [nnabla][INFO]: Initializing CPU extension...


DJ1


In [0]:

from __future__ import absolute_import
from six.moves import range
print("DJ1")

import functools
# Globally prepared params.
w = nn.parameter.get_parameter_or_create('BLURRING_FILTER', shape=(3,3), need_grad=False)  #why and how is it used for the parameter
w.d = np.array([[1., 2., 1.], [2., 4., 2.], [1., 2., 1.]])   

print('')


def lerp(a, b, t):
  print("Value of t is", t)
  if(t>0):
    return ((a + (b - a) * t))
  else:
    print("Only a")
    return a

def lerp_clip(a, b, t):
    return (a + (b - a) * F.clip_by_value(t, 0.0, 1.0))


#Computing the mean across the feature maps i.e. axis 2 and 3 for H and W with data format of x as NCHW
#--------------------
def compute_channel_mean(x, keepdims = True):
    ch_mean = F.mean(x, axis = [2,3], keepdims = keepdims)
    return ch_mean

#--------------------


#-------------------------------
#Computing the standard deviation across the feature maps i.e. axis 2 and 3 for H and W with data format of x as NCHW
def compute_channel_std(x, keepdims = True):
    x1 = compute_channel_mean(x)
    var = F.mean(F.pow_scalar((x - x1), 2.0), axis = [2,3], keepdims = keepdims)
    std = F.pow_scalar(var, 0.5)
    return std

#-------------------------------


#---------------------------------------------------
#Adaptive Instance Normalization to be used
def adain(x, y):
    y_s, y_b = y
    ch1_mean = compute_channel_mean(x)
    ch1_std = compute_channel_std(x)
    x = (x - ch1_mean) / ch1_std
    a1 = (y_s * x) + y_b
    return a1 

#-------------------------------


#New blur function to be used ------------------------------------------
def blur2d(x, f=[1,2,1], normalize = True, flip = False, stride = 1):
    ## assert x.ndim == 4 and all(dim.value is not None for dim in x.shape[1:])
    assert isinstance(stride, int) and stride >= 1

    f = np.array(f, dtype = np.float32)

    if f.ndim == 1:
        f = f[:, np.newaxis] * f[np.newaxis, :]

    if flip:
        f = f[::-1, ::-1]

    if normalize:
        f = f / np.sum(f)

    f = f[ np.newaxis, :, :]
    f = np.tile(f, [int(x.shape[1]), 1, 1])
    if f.shape == (1, 1) and f[0,0] == 1:
        return x

    #orig_dtype = x.dtype
    strides = (stride, stride) 
    w = nn.Variable.from_numpy_array(f)
    padding = (w.shape[1] // 2, w.shape[2] // 2)

    y = F.depthwise_convolution(x, w, pad = padding, stride = strides)
    return y

DJ1



In [0]:
def get_weight(he_std, gain = np.sqrt(2), use_wscale = False, lrmul = 1):
    if use_wscale:
        init_std = 1.0 / lrmul
        runtime_coeff = he_std * lrmul
    else: 
        init_std = he_std / lrmul
        runtime_coeff = lrmul

    w_init = I.NormalInitializer(sigma = init_std)   #I.NormalInitializer function is used with sigma argument as the standard deviation, so as to get the weights of these variables 
    return w_init, runtime_coeff

In [0]:
def downscale2d(x, factor = 2, gain = 1):
    #assert x.ndim == 4 and all(dim.value is not None for dim in x.shape[1:])
    assert isinstance(factor, int) and factor >=1

    if factor == 2:
        f = [np.sqrt(gain) / factor] * factor
        f = np.array(f, dtype = np.float32)
        if f.ndim == 1:
            f = f[:, np.newaxis] * f[np.newaxis, :]
        assert f.ndim == 2

        f = f[np.newaxis, :, :]
        f = np.tile(f, [int(x.shape[1]), 1, 1])
        w = nn.Variable.from_numpy_array(f)
        strides = (factor, factor)
        padding = (w.shape[1] // 2, w.shape[2] // 2)
        
        y = F.depthwise_convolution(x, w, stride = strides)
        return y 
    
    if gain != 1:
        x = x * gain

    if factor == 1:
        return x

    kernel = [factor, factor]
    y = F.average_pooling(x, kernel = kernel, stride = kernel)    # a bit of doubt regarding this statement
    return y

In [0]:
def dense(x, fmaps, gain = np.sqrt(2), use_wscale = False, lrmul = 1, **kwargs):
    he_std = I.calc_normal_std_he_forward(x.shape[1], fmaps)
    w_init, runtime_coeff = get_weight(he_std, gain, use_wscale, lrmul, **kwargs)
    w = nn.parameter.get_parameter_or_create('Dense', shape = (x.shape[1], fmaps), initializer = w_init, need_grad = True)
    w = w * runtime_coeff
    return (F.affine(x, w, bias=None, base_axis=1))

In [0]:
#Upscale operation function to be used 
def upscale2d(x, factor = 2, gain = 1):
    # to check whether the shape of the x is 4 or not
    #assert x.ndim == 4 and all(dim.value is not None for dim in x.shape[1:])
    assert isinstance(factor, int) and factor >=1 

    if gain!=1:
        x = x * gain
    
    if factor == 1:
        return x

    s = x.shape
    #reshape and tile function similar as that of the tensorflow
    x = F.reshape(x, [-1, s[1], s[2], 1, s[3], 1], False)
    x = F.tile(x, [1, 1, 1, factor, 1, factor])
    x = F.reshape(x, [-1, s[1], s[2] * factor, s[3] * factor])
    return x

In [0]:
def conv2d(x, fmaps, kernel, gain = np.sqrt(2), use_wscale = False, **kwargs):
  with nn.parameter_scope('Convolution'):
    assert isinstance(kernel, int) or isinstance(kernel, float)
    assert kernel >= 1 and kernel % 2 == 1
    if kernel == 1:
        padding = None
    else:
        padding = ((kernel - 2), (kernel - 2))  #tuple of padding
    kernel_tuple = (kernel, kernel)  #tuple of kernel_tuple
    he_std = I.calc_normal_std_he_forward(fmaps, x.shape[1], kernel_tuple)        # (kernel, kernel, x.shape[1], fmaps) (3, 2, 0, 1)
    w_init, runtime_coeff = get_weight(he_std, gain, use_wscale, **kwargs)
    return PF.convolution(x, fmaps, kernel = kernel_tuple, w_init = w_init, with_bias = False, pad = padding, apply_w = lambda w: w * runtime_coeff)


#usage of the Noise application which is used in the synthesis network.
def apply_noise(x, noise_var = None, randomize_noise = True):
    assert len(x.shape) == 4
    with nn.parameter_scope('Noise'):
        if noise_var is None or randomize_noise:
            noise = F.randn(shape = (x.shape[0], 1, x.shape[2], x.shape[3]))
        else:
            print("Dj1")
            noise = noise_var
        weight = nn.parameter.get_parameter_or_create(name = 'weight', shape = [x.shape[1]], initializer=I.ConstantInitializer(0))     # For Creating Random nnabla variable such that each entry follows the Normal Distribution..
        weight = F.reshape(weight, [1, -1, 1, 1], inplace= False)
        return (x + noise * weight)

In [0]:
def upscale2d_conv2d(x, fmaps, kernel, gain = np.sqrt(2), use_wscale = False, fused_scale = 'auto', **kwargs):
    assert kernel >= 1 and kernel % 2 == 1
    assert fused_scale in [True, False, 'auto']
    if fused_scale == 'auto':
        fused_scale = min(x.shape[2:]) * 2 >= 128

    if not fused_scale:
        return conv2d(upscale2d(x), fmaps, kernel, **kwargs)

    kernel_tuple = (kernel, kernel)
    he_std = I.calc_normal_std_he_forward(x.shape[1], fmaps, kernel_tuple)        # (kernel, kernel, x.shape[1], fmaps)  (2, 3, 0, 1)
    w_init, runtime_coeff = get_weight(he_std, gain, use_wscale, **kwargs)
    w = nn.parameter.get_parameter_or_create('deconv/W', shape = (x.shape[1], fmaps, kernel, kernel), initializer = w_init, need_grad = True)
    w = w * runtime_coeff
    w = F.pad(w, (1, 1, 1, 1), mode = 'constant')
    w = w[..., 1:, 1:] + w[..., :-1, 1:] + w[..., 1:, :-1] + w[..., :-1, :-1]
    kernel_size = w.shape[-1]
    padding = ((kernel_size - 2) / 2, (kernel_size - 2) / 2)
    stride = (2 , 2)
    return F.deconvolution(x, w, bias = None, pad = padding, stride = stride)

In [0]:
# Change in the apply_bias function.
def apply_bias(x, lrmul = 1):
    b = nn.parameter.get_parameter_or_create(name = 'bias', shape = [x.shape[1]], initializer = I.ConstantInitializer(0))   # Change the initializer to either float or convert the datatype to float32.
    b = b * lrmul  # No Modifications requried..
    if x.ndim == 2:
        return (x + F.reshape(b, [1, -1]))
    return (x + F.reshape(b, [1, -1, 1, 1])) 

In [0]:
#defining the style modulation i.e. combining ys and yb with the resultant of the addition of the noise and the feature maps
def style_mod(x, dlatent, **kwargs):
    with nn.parameter_scope('StyleMod'):
        style = apply_bias(dense(dlatent, fmaps = x.shape[1] * 2, **kwargs))
        style = F.reshape(style, [-1, 2, x.shape[1]] + [1] * (len(x.shape) - 2))
        return x * (style[:,0] + 1) + style[:,1]


In [0]:
def conv2d_downscale2d(x, fmaps, kernel, gain = np.sqrt(2), use_wscale = False, fused_scale = 'auto', **kwargs):
    #assert kernel % 2 == 1 and kernel >= 1
    assert fused_scale in [True, False, 'auto']
    if fused_scale == 'auto':
        fused_scale = min(x.shape[2:]) >= 128   #why is this written and what is the condition behind that

    if not fused_scale:
        return downscale2d(conv2d(x, fmaps, kernel, **kwargs))

    kernel_tuple = (kernel, kernel)
    he_std = I.calc_normal_std_he_forward(fmaps, x.shape[1], kernel_tuple)       # (kernel, kernel, x.shape[1], fmaps) (3, 2, 0, 1)
    w_init, runtime_coeff = get_weight(he_std, gain = gain, use_wscale = use_wscale, **kwargs)
    w = nn.parameter.get_parameter_or_create('conv/W', shape = (fmaps, x.shape[1], kernel, kernel), initializer = w_init, need_grad = True)
    w = w * runtime_coeff
    w = F.pad(w, (1, 1, 1, 1), mode = 'constant')
    w = w[..., 1:, 1:] + w[..., :-1, 1:] + w[..., 1:, :-1] + w[..., :-1, :-1]
    w = w * 0.25
    kernel_size = w.shape[-1]
    padding = ((kernel_size - 2) / 2, (kernel_size - 2) / 2)
    stride = (2,2)
    return F.convolution(x, w, bias=None, pad = padding, stride = stride)

In [0]:
def minibatch_stddev_layer(x, group_size=4, num_new_features=1):
    group_size = min(group_size, x.shape[0])     # Minibatch must be divisible by (or smaller than) group_size.
    s = x.shape                                             # [NCHW]  Input shape.
    y = F.reshape(x, [group_size, -1, num_new_features, s[1]//num_new_features, s[2], s[3]])   # [GMncHW] Split minibatch into M groups of size G. Split channels into n channel groups c.
    y -= F.mean(y, axis=0, keepdims=True)           # [GMncHW] Subtract mean over group.
    y = F.mean(F.pow_scalar(y, 2.0), axis=0)                # [MncHW]  Calc variance over group.
    y = F.add_scalar(F.pow_scalar(y, 2.0) + 1e-8)                                   # [MncHW]  Calc stddev over group.
    y = F.mean(y, axis=[2,3,4], keepdims=True)      # [Mn111]  Take average over fmaps and pixels.
    y = F.mean(y, axis=[2])                         # [Mn11] Split channels into c channel groups
    y = F.tile(y, [group_size, 1, s[2], s[3]])             # [NnHW]  Replicate over group and pixels.
    return F.concatenate(x, y, axis=1)

In [0]:
#--------------------------------------
#application of the activation function
def leaky_relu(x):
    return functools.partial(F.leaky_relu, alpha = 0.2)(x)
#--------------------------------------


In [0]:
#---------------------------------------------
# defining the pixel normalization which is probably used in the Discriminator network
def pixel_norm(x, epsilon = 1e-8):
    mean1 = F.mean(F.pow_scalar(x, 2.0), axis = 1, keepdims = True)
    new_mean1 = F.add_scalar(mean1, epsilon)
    denominator = F.pow_scalar(new_mean1, 0.5)
    x = F.div2(x, denominator)
    return x

#--------------------------------------------------

In [0]:
#---------------------------------------------------------------
#Defining the Instance Normalization which is used in the Synthesis Generator Network with the format NCHW of the x   Change this FUnction i.e. Instance Normalization..
def instance_norm(x, epsilon = 1e-8):
    assert len(x.shape) == 4
    with nn.parameter_scope('InstanceNorm'):
        x1 = compute_channel_mean(x)
        x = x - x1
        x2 = compute_channel_mean(F.add_scalar(F.pow_scalar(x , 2.0), epsilon))
        x2 = F.pow_scalar(x2, 0.5)
        x = x / x2
        return x



In [0]:
def G1(
    dlatents_in,
    dlatent_size = 512,
    num_channels = 3,
    resolution = 256,
    fmap_base = 8192,
    fmap_decay = 1.0,
    fmap_max = 512,
    use_style = True,
    const_input_layer = True,
    use_noise = True,
    randomize_noise = True,
    nonlinearity = 'lrelu',
    use_wscale = True,
    use_pixel_norm = False,
    use_instance_norm = True,
    dtype = 'float32',
    fused_scale = True,
    blur_filter = [1,2,1],
    structure = 'linear',
    is_template_graph = False,
    force_clean_graph = False,
    **_kwargs):
    
    with nn.parameter_scope('G_synthesis'):
        resolution_log2 = int(np.log2(resolution))
        assert resolution>=4 and resolution == 2 ** (resolution_log2)

        def nf(stage):
            return min(int(fmap_base / (2.0 ** (stage * fmap_decay))), fmap_max)

        def blur(x):
            if blur_filter:
                return blur2d(x, blur_filter)
            else:
                return x

        if is_template_graph:
            force_clean_graph = True

        if force_clean_graph:
            randomize_noise = False

        if structure == 'auto':
            if force_clean_graph:
                structure = 'linear'
            else:
                structure = 'recursive'
        #structure = 'linear' if force_clean_graph else 'recursive'

        act, gain = {'relu' : (F.relu, np.sqrt(2)), 'lrelu' : (leaky_relu, np.sqrt(2))}[nonlinearity]
        num_layers = 2 * resolution_log2 - 2
    
        if use_style:
            num_styles = num_layers
        else:
            num_styles = 1

        images_out = None

        lod_in = 0

        noise_inputs = []
        if use_noise:
            for layer_idx in range(num_layers):
                res = layer_idx // 2 + 2
                shape = [1, use_noise, 2 ** (res), 2 ** (res)]
                noise_inputs.append(nn.parameter.get_parameter_or_create('noise{}'.format(layer_idx), shape = shape, initializer = I.NormalInitializer(), need_grad = True))
        
        a1 = []
        a11 = []
        a12 = []
        a2 = []
        a3 = []
        a4 = []
        a5 = []
        a6 = []
        a71 = []
        a72 = []
        a8 = []
        a9 = []
        a10 = []
        
        a1.append(dlatents_in)

        def layer_epilogue(x, a2, a3, a4, a5, a6, layer_idx):
            if use_noise:
                x = apply_noise(x, noise_inputs[layer_idx], randomize_noise = randomize_noise, **_kwargs)
                a2.append(x)
            x = apply_bias(x)
            a3.append(x)
            x = act(x)
            a4.append(x)
            if use_pixel_norm:
                x = pixel_norm(x)
                print("DJ2")
            if use_instance_norm:
                x = instance_norm(x)
                a5.append(x)
            if use_style:
                x = style_mod(x, dlatents_in[:, layer_idx], use_wscale = use_wscale)
                a6.append(x)
            return x

        def torgb(res, a71, a72, x): # res = 2..resolution_log2
            lod = resolution_log2 - res
            with nn.parameter_scope('ToRGB_lod{}'.format(lod)):
                x = conv2d(x, fmaps=num_channels, kernel=1, gain=1, use_wscale=use_wscale)
                a71.append(x)
                x = apply_bias(x)
                a72.append(x)
                return x

        def block(res, a1, a2, a3, a4, a5, a6, a11, a12, x): # res = 3..resolution_log2                                      # Change the block function....
            with nn.parameter_scope('{}x{}'.format(2**res, 2**res)):
                with nn.parameter_scope('Conv0_up'):
                    x = upscale2d_conv2d(x, fmaps=nf(res-1), kernel=3, gain=gain, use_wscale=use_wscale, fused_scale=fused_scale)
                    a11.append(x)
                    x = blur2d(x)
                    a12.append(x)
                    x = layer_epilogue(x, a2, a3, a4, a5, a6, res*2-4)
                with nn.parameter_scope('Conv1'):
                    x = conv2d(x, fmaps=nf(res-1), kernel = 3, gain=gain, use_wscale=use_wscale)
                    a1.append(x)
                    x = layer_epilogue(x, a2, a3, a4, a5, a6, res*2-3)
            return x

        with nn.parameter_scope('4 X 4'):
            if const_input_layer:
                with nn.parameter_scope('Const'):
                    x = nn.parameter.get_parameter_or_create(name = 'const', shape = [1, nf(1), 4, 4], initializer = I.ConstantInitializer(value = 1))
                    a1.append(x)
                    x = layer_epilogue(F.tile(x, [dlatents_in.shape[0], 1, 1, 1]), a2, a3, a4, a5, a6, 0)
            else:
                with nn.parameter_scope('Dense'):
                    x = dense(dlatents_in[:, 0], fmaps=nf(1)*16, gain=gain/4, use_wscale=use_wscale) 
                    x = F.reshape(x, [-1, nf(1), 4, 4])
                    x = layer_epilogue(x, a2, a3, a4, a5, a6, layer_idx=0)
            with nn.parameter_scope('Conv'):
                x = conv2d(x, fmaps=nf(1), kernel=3, gain=gain, use_wscale=use_wscale)
                a1.append(x)
                x = layer_epilogue(x, a2, a3, a4, a5, a6, 1)

        if structure == 'fixed':
            for res in range(3, resolution_log2 + 1):
                x = block(res, a1, a2, a3, a4, a5, a6, x)
                images_out = torgb(resolution_log2, a7, x)

        if structure == 'linear':
            images_out = torgb(2, a71, a72, x)
            a10.append(images_out)
            for res in range(3, resolution_log2 + 1):
                l1 = resolution_log2 - res
                lod = l1
                x = block(res, a1, a2, a3, a4, a5, a6, a11, a12, x)
                img = torgb(res, a71, a72, x)
                images_out = upscale2d(images_out)
                with nn.parameter_scope('Grow_lod%d' % l1):
                    images_out1 = lerp(img, images_out, lod_in- lod)
                    a10.append(images_out1)


        a8.append(a1)
        a8.append(a11)
        a8.append(a12)
        a8.append(a2)
        a8.append(a3)
        a8.append(a4)
        a8.append(a5)
        a8.append(a6)
        a8.append(a71)
        a8.append(a72)
        a8.append(a10)

        return a8, F.identity(images_out1)


In [0]:
import pickle
with open("noise.txt", "rb") as fp:   
   noise = pickle.load(fp)

In [0]:
with open("const.txt", "rb") as fp:   
   const = pickle.load(fp)

In [0]:
with open("input1.txt", "rb") as fp:   
   input1 = pickle.load(fp)

In [0]:
with open("initial_conv_weights1.txt", "rb") as fp:   
   initial_conv_weights1 = pickle.load(fp)

In [0]:
with open("initial_conv_weights2.txt", "rb") as fp:   
   initial_conv_weights2 = pickle.load(fp)

In [0]:
with open("initial_conv_weights3.txt", "rb") as fp:   
   initial_conv_weights3 = pickle.load(fp)

In [0]:
with open("initial_conv_weights4.txt", "rb") as fp:   
   initial_conv_weights4 = pickle.load(fp)

In [0]:
with open("initial_conv_weights5.txt", "rb") as fp:   
   initial_conv_weights5 = pickle.load(fp)

In [0]:
with open("initial_conv_weights6.txt", "rb") as fp:   
   initial_conv_weights6 = pickle.load(fp)

In [0]:
with open("initial_conv_weights7.txt", "rb") as fp:   
   initial_conv_weights7 = pickle.load(fp)

In [0]:
input12 = input1[0]
input12 = nn.Variable.from_numpy_array(input12)

In [0]:
# Printing the Inputs:
print(input12[0][0][0].d)
print(input12[0][0][1].d)
print(input12[0][0][2].d)
print(input12[0][0][3].d)


0.83128
0.2854586
0.9196952
0.8687711


In [0]:
list1, output1 = G1(input12, num_channels = 3, resolution= 256, randomize_noise = False, use_wscale = False, fused_scale=False, structure='linear')

Dj1
Dj1
Dj1
Dj1
Value of t is -5
Only a
Dj1
Dj1
Value of t is -4
Only a
Dj1
Dj1
Value of t is -3
Only a
Dj1
Dj1
Value of t is -2
Only a
Dj1
Dj1
Value of t is -1
Only a
Dj1
Dj1
Value of t is 0
Only a


In [0]:
ax = nn.get_parameters()

In [0]:
l = 0 
for i, j in ax.items():
    if(l==14):
        break
    j.data.copy_from(nn.Variable.from_numpy_array(noise[l]).data)
    l = l + 1

In [0]:
l = 0
l1 = 0
for i,j in ax.items():
    if(l==19):
        break
    if(l>=14):
        j.data.copy_from(nn.Variable.from_numpy_array(const[l1]).data)
        l1 = l1 + 1
    l = l + 1

In [0]:
l = 0
l1 = 0
for i,j in ax.items():
    if(l==26):
        break
    if(l>=19):
        if(l==19 or l==24):
            a1 = np.transpose(initial_conv_weights1[l1], axes = [3, 2, 0, 1])
            j.data.copy_from((nn.Variable.from_numpy_array(a1)).data) 
        else:
            j.data.copy_from(nn.Variable.from_numpy_array(initial_conv_weights1[l1]).data)         
        l1 = l1 + 1
    l = l + 1

In [0]:
l = 0
l1 = 0
for i,j in ax.items():
    if(l==38):
        print("value of l is:", l)
        break
    if(l>=26):
        if(l==26):
            a1 = np.transpose(initial_conv_weights2[l1], axes = [3, 2, 0, 1])
            j.data.copy_from(nn.Variable.from_numpy_array(a1).data) 
        elif(l==31):
            a1 = np.transpose(initial_conv_weights2[l1], axes = [3, 2, 0, 1])
            j.data.copy_from(nn.Variable.from_numpy_array(a1).data)
        elif(l==36):
            a1 = np.transpose(initial_conv_weights2[l1], axes = [3, 2, 0, 1])
            j.data.copy_from(nn.Variable.from_numpy_array(a1).data)
        else:
            j.data.copy_from(nn.Variable.from_numpy_array(initial_conv_weights2[l1]).data)         
        l1 = l1 + 1
    l = l + 1

value of l is: 38


In [0]:
l = 0
l1 = 0
for i,j in ax.items():
    if(l==50):
        print("value of l is:", l)
        break
    if(l>=38):
        if(l==38):
            a1 = np.transpose(initial_conv_weights3[l1], axes = [3, 2, 0, 1])
            j.data.copy_from(nn.Variable.from_numpy_array(a1).data) 
        elif(l==43 or l==48):
            a1 = np.transpose(initial_conv_weights3[l1], axes = [3, 2, 0, 1])
            j.data.copy_from(nn.Variable.from_numpy_array(a1).data)
        else:
            j.data.copy_from(nn.Variable.from_numpy_array(initial_conv_weights3[l1]).data)         
        l1 = l1 + 1
    l = l + 1

value of l is: 50


In [0]:
l = 0
l1 = 0
for i,j in ax.items():
    if(l==62):
        print("value of l is:", l)
        break
    if(l>=50):
        if(l==50):
            a1 = np.transpose(initial_conv_weights4[l1], axes = [3, 2, 0, 1])
            j.data.copy_from(nn.Variable.from_numpy_array(a1).data) 
        elif(l==55 or l==60):
            a1 = np.transpose(initial_conv_weights4[l1], axes = [3, 2, 0, 1])
            j.data.copy_from(nn.Variable.from_numpy_array(a1).data)
        else:
            j.data.copy_from(nn.Variable.from_numpy_array(initial_conv_weights4[l1]).data)         
        l1 = l1 + 1
    l = l + 1

value of l is: 62


In [0]:
l = 0
l1 = 0
for i,j in ax.items():
    if(l==74):
        print("value of l is:", l)
        break
    if(l>=62):
        if(l==62):
            a1 = np.transpose(initial_conv_weights5[l1], axes = [3, 2, 0, 1])
            j.data.copy_from(nn.Variable.from_numpy_array(a1).data) 
        elif(l==67 or l==72):
            a1 = np.transpose(initial_conv_weights5[l1], axes = [3, 2, 0, 1])
            j.data.copy_from(nn.Variable.from_numpy_array(a1).data)
        else:
            j.data.copy_from(nn.Variable.from_numpy_array(initial_conv_weights5[l1]).data)         
        l1 = l1 + 1
    l = l + 1

value of l is: 74


In [0]:
l = 0
l1 = 0
for i,j in ax.items():
    if(l==86):
        print("value of l is:", l)
        break
    if(l>=74):
        if(l==74):
            a1 = np.transpose(initial_conv_weights6[l1], axes = [3, 2, 0, 1])
            j.data.copy_from(nn.Variable.from_numpy_array(a1).data) 
        elif(l==79):
            a1 = np.transpose(initial_conv_weights6[l1], axes = [3, 2, 0, 1])
            j.data.copy_from(nn.Variable.from_numpy_array(a1).data)
        elif(l==84):
            a1 = np.transpose(initial_conv_weights6[l1], axes = [3, 2, 0, 1])
            j.data.copy_from(nn.Variable.from_numpy_array(a1).data)
        else:
            j.data.copy_from(nn.Variable.from_numpy_array(initial_conv_weights6[l1]).data)         
        l1 = l1 + 1
    l = l + 1

value of l is: 86


In [0]:
l = 0
l1 = 0
for i,j in ax.items():
    if(l==98):
        print("value of l is:", l)
        break
    if(l>=86):
        if(l==86):
            a1 = np.transpose(initial_conv_weights7[l1], axes = [3, 2, 0, 1])
            j.data.copy_from(nn.Variable.from_numpy_array(a1).data) 
        elif(l==91 or l==96):
            a1 = np.transpose(initial_conv_weights7[l1], axes = [3, 2, 0, 1])
            j.data.copy_from(nn.Variable.from_numpy_array(a1).data)
        else:
            j.data.copy_from(nn.Variable.from_numpy_array(initial_conv_weights7[l1]).data)         
        l1 = l1 + 1
    l = l + 1

In [0]:
list2, output2 = G1(input12, num_channels = 3, resolution= 256, randomize_noise = False, use_wscale = False, fused_scale=False, structure='linear')


Dj1
Dj1
Dj1
Dj1
Value of t is -5
Only a
Dj1
Dj1
Value of t is -4
Only a
Dj1
Dj1
Value of t is -3
Only a
Dj1
Dj1
Value of t is -2
Only a
Dj1
Dj1
Value of t is -1
Only a
Dj1
Dj1
Value of t is 0
Only a


In [0]:
# Printing the Outputs:
print(output2[0][0][0][0].d)
print(output2[0][0][0][1].d)
print(output2[0][0][0][2].d)
print(output2[0][0][0][3].d)
print()
print(output2[0][0][1][0].d)
print(output2[0][0][1][1].d)
print(output2[0][0][1][2].d)
print(output2[0][0][1][3].d)
print()
print(output2[0][0][2][0].d)
print(output2[0][0][2][1].d)
print(output2[0][0][2][2].d)
print(output2[0][0][2][3].d)

11.214745
-12.6091
-23.258469
-49.759506

22.246475
0.50022924
-3.6949687
-40.752403

55.498444
43.97695
13.625519
-38.80551


In [0]:
print(output2.d)

[[[[ 1.12147446e+01 -1.26091003e+01 -2.32584686e+01 ...  1.06867874e+02
     7.76745300e+01  3.43050308e+01]
   [ 2.22464752e+01  5.00229239e-01 -3.69496870e+00 ...  1.77655380e+02
     1.37520386e+02  4.82230873e+01]
   [ 5.54984436e+01  4.39769516e+01  1.36255188e+01 ...  1.62914917e+02
     1.57168686e+02  8.39401779e+01]
   ...
   [ 4.36366272e+01  6.75754318e+01  2.07843246e+01 ... -1.19093681e+02
    -1.15214134e+02 -1.01103683e+02]
   [ 1.91865234e+01  4.95323715e+01  4.82497482e+01 ... -1.17203270e+02
    -1.32540405e+02 -1.13222588e+02]
   [ 5.55048037e+00  3.08207169e+01  4.63837051e+01 ... -1.45084747e+02
    -1.49820435e+02 -9.18438950e+01]]

  [[-5.28250923e+01 -8.65906830e+01 -8.80172729e+01 ... -3.66232777e+00
    -3.00698757e+01 -4.28526459e+01]
   [-4.97590294e+01 -7.17598267e+01 -6.79165497e+01 ...  2.75078411e+01
     1.57569742e+01 -2.65963745e+01]
   [-1.87806778e+01 -2.57676392e+01 -6.90390320e+01 ...  8.95478535e+00
     2.10470390e+01  5.62057972e+00]
   ...
   